## **soynlp**

In [1]:
import re
import pandas as pd
import numpy as np

from collections import Counter
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

path = 'C:/VSC/JJaemni/CSV/'

In [2]:
df = pd.read_csv(path + 'wanted_jobplanet_2.csv')
df.head()

,Title,Company,Content,Requirement,Preference,StackTool,Link,Location,label
0,[100억↑투자] 데이터사이언티스트,해줌,"해줌은 2012년에 설립된 회사로, 소비자로부터 멀게만 느껴졌던 대한민국 태양광 산...",\n- 관련 업무 경력 3년 이상인 분\n- 시계열 데이터를 분석하여 예측에 대한 ...,\n- 기상관련 데이터 경험\n- 머신러닝 관련 모델링 도구 경험자 (Tensorf...,\nPytorch\nTensorflow\nPython,https://www.wanted.co.kr/wd/147049,서울,데이터 사이언티스트
1,주니어 블록체인 데이터 사이언티스트,클로인트,클로인트(Kloint)는 디지털자산에 대한 다양한 위협으로부터 사회와 대중을 보호하...,"\n• 데이터 사이언스 관련 기업 근무, 연구활동 등 업무 수행 경험\n• 블록체인...",\n• 블록체인 및 암호화폐 관련 기업 근무 경력\n• 빅데이터 관련 기업 근무 경...,\nHadoop\nMongoDB\nMySQL\nSpark\nPython\nR\nTa...,https://www.wanted.co.kr/wd/174917,서울,데이터 사이언티스트
2,시니어 블록체인 데이터 사이언티스트,클로인트,클로인트(Kloint)는 디지털자산에 대한 다양한 위협으로부터 사회와 대중을 보호하...,\n• 데이터 사이언스 관련 실무 경력 7년 이상 또는 그에 준하는 역량 보유\n•...,\n• 블록체인 및 암호화폐 관련 기업 근무 경력\n• 빅데이터 관련 기업 근무 경...,\nHadoop\nMongoDB\nMySQL\nSpark\nPython\nR\nTa...,https://www.wanted.co.kr/wd/174913,서울,데이터 사이언티스트
3,데이터사이언티스트 (AI추천엔진팀),씨제이올리브영(CJ올리브영),"All Live Better,\n\nNo.1 헬스&뷰티 스토어 올리브영은 20년 이...","\n관련 업무 경력 5~10년 (데이터 상용화 모델링, 개발역량 등)\nMachin...","\n• 관련학과(수학, 통계, 전산, 데이터 분석) 전공자 우대\n• ML 관련 서...",\nAzure\nAWS\nGCP,https://www.wanted.co.kr/wd/164003,서울,데이터 사이언티스트
4,데이터 사이언티스트,씨제이대한통운(CJ대한통운),CJ대한통운은 글로벌 리딩 100년 기업으로의 도약을 목표로 성장하고 있는 종합 물...,\n[경력사항]\n• 데이터 모델링 경력 5년 이상\n\n,\n• 데이터 시각화 도구(Tableau 등) 사용이 가능하신 분\n• ML 프레임...,\nGitLab\nCSS\nTypeScript\nReact.js\nNext.js,https://www.wanted.co.kr/wd/163197,서울,데이터 사이언티스트


- 수집한 데이터는 채용공고이므로 띄어쓰기가 잘 되어있고, 오타가 거의 없을거라고 생각함

In [11]:
Req = df.Requirement.tolist()

In [16]:
Req[0]

'\n- 관련 업무 경력 3년 이상인 분\n- 시계열 데이터를 분석하여 예측에 대한 경험이 있는 분\n- 논문 등 알고리즘에 대해 분석 후 구현 경험이 있는 분\n- 통계분석, 인공지능/머신러닝 관련 석/박사학위 또는 이에 준하는 경력을 소지하신 분\n- Python 언어 사용 가능\n'

In [38]:
Req2 = []
for idx in range(0, df.shape[0]):
    str = Req[idx]
    new_str = re.sub('\n', '', str)
    new_str2 = re.sub(r'[^\uAC00-\uD7A30-9\s]', ' ', new_str)
    new_str3 = re.split(' ', new_str2)
    new_str4 = ' '.join(filter(None, new_str3))
    Req2.append(new_str4)
    print(f'{idx}. {Req2[idx]}')

0. 관련 업무 경력 3년 이상인 분 시계열 데이터를 분석하여 예측에 대한 경험이 있는 분 논문 등 알고리즘에 대해 분석 후 구현 경험이 있는 분 통계분석 인공지능 머신러닝 관련 석 박사학위 또는 이에 준하는 경력을 소지하신 분 언어 사용 가능
1. 데이터 사이언스 관련 기업 근무 연구활동 등 업무 수행 경험 블록체인 관련 기업 근무 연구활동 등 업무 수행 경험 데이터 분석 및 머신러닝 관련 언어 등 활용 역량 온체인 데이터 분석에 대한 이해도 보유자 선호
2. 데이터 사이언스 관련 실무 경력 7년 이상 또는 그에 준하는 역량 보유 대용량 데이터 분석 및 머신러닝 관련 언어 등 활용 능력 대용량 데이터 분석을 통해 문제를 정의하고 해결할 수 있는 능력 관련 알고리즘에 대한 논문 작성 등 분석 후 구현 경험 블록체인에 대한 비지니스적 기술적 이해도 보유자 선호 관련 실무경험 보유자 선호
3. 관련 업무 경력 5 10년 데이터 상용화 모델링 개발역량 등 활용 데이터 분석 가능 개발 가능자다양한 분석 툴 등 활용 가능고객 상품 브랜드 분석 및 비즈니스 관련 인사이트 도출 경험
4. 경력사항 데이터 모델링 경력 5년 이상
5. 관련 분야 경력 4년 이상이신 분 이론에 대한 전반적인 이해가 있으신 분 딥러닝 프레임워크 등 를 활용한 모델 개발 및 배포 경험이 있으신 분 대용량 데이터를 분석 및 시각화해본 경험이 있으신 분 해외 출장에 결격 사유가 없는 분 팀원들과 적극적으로 소통할 수 있는 분 주도적으로 업무를 이끌어가시는 분 제출서류 자율 양식의 이력서 필수 포함내용 성향 및 업무 가치관 개발경험 성공 및 실패 경험 협업 프로젝트 진행 관련 개발 성과 협업 경험을 보여줄 수 있는 포트폴리오
6. 데이터 사이언티스트로 이런 분을 찾아요 3년 이상의 모델링에 기반한 프로젝트 실무 경험이 있으신 분 개인화 추천 및 고객 경험 관련 모델링 경험이 있으신 분 이커머스 및 거래 데이터 도메인에 대한 이해도 혹은 관련 근무 경험이 있으신 분 과 프로그래밍을 능숙하게 다룰 수 있으

In [39]:
req = pd.DataFrame({
    'req' : Req2
})
req.head()

,req
0,관련 업무 경력 3년 이상인 분 시계열 데이터를 분석하여 예측에 대한 경험이 있는 ...
1,데이터 사이언스 관련 기업 근무 연구활동 등 업무 수행 경험 블록체인 관련 기업 근...
2,데이터 사이언스 관련 실무 경력 7년 이상 또는 그에 준하는 역량 보유 대용량 데이...
3,관련 업무 경력 5 10년 데이터 상용화 모델링 개발역량 등 활용 데이터 분석 가능...
4,경력사항 데이터 모델링 경력 5년 이상


In [47]:
req.to_csv('/VSC/JJaemni/CSV/req.csv', index=False, encoding='utf-8-sig')

In [48]:
sents = DoublespaceLineCorpus('/VSC/JJaemni/CSV/req.csv', iter_sent=True)
noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(sents)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1468 from 182 sents. mem=0.167 Gb                    


[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=6854, mem=0.169 Gb
[Noun Extractor] batch prediction was completed for 500 words
[Noun Extractor] checked compounds. discovered 26 compounds
[Noun Extractor] postprocessing detaching_features : 251 -> 251
[Noun Extractor] postprocessing ignore_features : 251 -> 243
[Noun Extractor] postprocessing ignore_NJ : 243 -> 242
[Noun Extractor] 242 nouns (26 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.170 Gb                    
[Noun Extractor] 65.07 % eojeols are covered


In [57]:
nouns

{'오케스트레이션': NounScore(frequency=2, score=1.0),
 '데이터베이스': NounScore(frequency=17, score=1.0),
 '사이언티스트': NounScore(frequency=3, score=1.0),
 '커뮤니케이션': NounScore(frequency=34, score=1.0),
 '엔지니어링': NounScore(frequency=14, score=1.0),
 '프레임워크': NounScore(frequency=11, score=1.0),
 '소프트웨어': NounScore(frequency=7, score=1.0),
 '파이프라인': NounScore(frequency=20, score=1.0),
 '프로그래밍': NounScore(frequency=20, score=1.0),
 '데이터분석': NounScore(frequency=3, score=0.42857142857142855),
 '엔지니어': NounScore(frequency=15, score=1.0),
 '의사결정': NounScore(frequency=7, score=1.0),
 '알고리즘': NounScore(frequency=13, score=1.0),
 '프로세스': NounScore(frequency=8, score=1.0),
 '머신러닝': NounScore(frequency=20, score=1.0),
 '비즈니스': NounScore(frequency=20, score=1.0),
 '프로덕트': NounScore(frequency=7, score=1.0),
 '아키텍처': NounScore(frequency=6, score=1.0),
 '요구사항': NounScore(frequency=3, score=1.0),
 '블록체인': NounScore(frequency=4, score=1.0),
 '인사이트': NounScore(frequency=16, score=1.0),
 '스타트업': NounScore(frequency=3, sc

In [59]:
nouns2 = {}
for noun, score in nouns.items():
    if len(noun) > 1:
        nouns2[noun] = score
nouns2

{'오케스트레이션': NounScore(frequency=2, score=1.0),
 '데이터베이스': NounScore(frequency=17, score=1.0),
 '사이언티스트': NounScore(frequency=3, score=1.0),
 '커뮤니케이션': NounScore(frequency=34, score=1.0),
 '엔지니어링': NounScore(frequency=14, score=1.0),
 '프레임워크': NounScore(frequency=11, score=1.0),
 '소프트웨어': NounScore(frequency=7, score=1.0),
 '파이프라인': NounScore(frequency=20, score=1.0),
 '프로그래밍': NounScore(frequency=20, score=1.0),
 '데이터분석': NounScore(frequency=3, score=0.42857142857142855),
 '엔지니어': NounScore(frequency=15, score=1.0),
 '의사결정': NounScore(frequency=7, score=1.0),
 '알고리즘': NounScore(frequency=13, score=1.0),
 '프로세스': NounScore(frequency=8, score=1.0),
 '머신러닝': NounScore(frequency=20, score=1.0),
 '비즈니스': NounScore(frequency=20, score=1.0),
 '프로덕트': NounScore(frequency=7, score=1.0),
 '아키텍처': NounScore(frequency=6, score=1.0),
 '요구사항': NounScore(frequency=3, score=1.0),
 '블록체인': NounScore(frequency=4, score=1.0),
 '인사이트': NounScore(frequency=16, score=1.0),
 '스타트업': NounScore(frequency=3, sc

In [68]:
nouns3 = sorted(nouns2.items(), key=lambda x: x[1].frequency)
for noun, score in nouns3:
    print(f'{noun}: freq={score.frequency}')

알아: freq=1
의지: freq=1
설득: freq=1
정확: freq=1
데이터사이언티스트: freq=1.0
이상데이터베이스: freq=1.0
데이터분석도구: freq=1.0
가능적극적: freq=1.0
작성가능: freq=1.0
데이터처리: freq=1.0
이상보유: freq=1.0
문제정의: freq=1.0
분석언어: freq=1.0
분석결과: freq=1.0
연구개발: freq=1.0
기술통계: freq=1.0
데이터구조: freq=1.0
이상데이터: freq=1.0
협업능력: freq=1.0
데이터수집: freq=1.0
데이터관리: freq=1.0
통계분석: freq=1.0
개발역량: freq=1.0
개발경험: freq=1.0
분석모델: freq=1.0
가설검증: freq=1.0
분석방법: freq=1.0
이상학위: freq=1.0
오케스트레이션: freq=2
이커머스: freq=2
신호처리: freq=2
모니터링: freq=2
졸업자: freq=2
마케팅: freq=2
병무청: freq=2
분석적: freq=2
선착순: freq=2
예정자: freq=2
디자인: freq=2
고도화: freq=2
부서: freq=2
공유: freq=2
소스: freq=2
원리: freq=2
창출: freq=2
독해: freq=2
영문: freq=2
절차: freq=2
형태: freq=2
근거: freq=2
지향: freq=2
내용: freq=2
유리: freq=2
여러: freq=2
뛰어: freq=2
좋아: freq=2
효율: freq=2
혁신: freq=2
패턴: freq=2
작업: freq=2
사이언티스트: freq=3
데이터분석: freq=3
요구사항: freq=3
스타트업: freq=3
박사학위: freq=3
도메인: freq=3
팀원들: freq=3
개발자: freq=3
소지자: freq=3
트렌드: freq=3
유연: freq=3
꼼꼼: freq=3
소지: freq=3
제시: freq=3
적응: freq=3
전반: freq=3
산업: freq=3
추론

## **de 나누기**

In [2]:
import re
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

path = 'C:/VSC/JJaemni/CSV/'

In [3]:
requir = pd.read_csv(path + 'requirement_na.csv')
prefer = pd.read_csv(path + 'preference_na.csv')

In [6]:
requir_list = requir.DE.tolist()

In [8]:
print(requir_list)

['Tableau', 'ArgoCD', '가능', '결함', '역할', '교육', '규칙', 'ANSI', '면제자', '사유', 'Oozie', 'Dagster', '클러스터', '프론트', 'Unix', '물리', '긍정', '시간', 'HDinsights', '경력', 'RAC', '엑', 'LLM', '의식', '분산', '모델링', 'Docker', '모델', '야간', 'streaming', '경력자', 'FastAPI', 'Spark', 'MFC', 'Flume', 'Presto', '세', '부품', '아래', '운용', 'Infra', '프로덕트', '생물학', 'Script', '하이퍼', '유지', '워크플로우', '중요', '홈', 'SAS', '경력직', 'git', '개', 'Framework', '제안', '관습', '레이크하우스', '학위자', '택스', '최적', '경험', 'native', '평가', 'JOIN', 'PostgreSQL', 'UI', '설득력', 'DAG', 'PPDM', '전기', '우대', '공모전', '발굴', '운전면허', '관심', '논리', '해당자', '파악', '박사', '원천', '대시', '스토리보드', 'DE', '유선', 'Spring', '능숙자', '이행', '고', 'OpenCV', '고등학교', '기본', '프로토타입', 'RESTFul', '학위', 'jsp', '의지', '리뷰', '이상자', '상급', '확정', 'workflow', '동료', '양식', '지금', '의사', '대학', '툴', '기관', 'IDPA', 'NW', '프로토콜', '선임', '슈팅', '프로젝트', '응용', '졸업', '혁신성', 'Angular', '간호학', '입사일', '컨텐츠', '물리적', 'Migration', 'Bash', 'Replica', '일정', '가지', '중심', '입사', '채용', 'WAF', 'Jenkins', '능통자', 'Hive', '플랫폼', '컴퓨터', '측정

## **Okt**

In [1]:
import re
import pandas as pd
import numpy as np

from konlpy.tag import Okt
from collections import Counter

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

path = 'C:/VSC/JJaemni/CSV/'

In [2]:
df = pd.read_csv(path + 'wanted_jobplanet_2.csv')
df.head()

,Title,Company,Content,Requirement,Preference,StackTool,Link,Location,label
0,[100억↑투자] 데이터사이언티스트,해줌,"해줌은 2012년에 설립된 회사로, 소비자로부터 멀게만 느껴졌던 대한민국 태양광 산...",\n- 관련 업무 경력 3년 이상인 분\n- 시계열 데이터를 분석하여 예측에 대한 ...,\n- 기상관련 데이터 경험\n- 머신러닝 관련 모델링 도구 경험자 (Tensorf...,\nPytorch\nTensorflow\nPython,https://www.wanted.co.kr/wd/147049,서울,데이터 사이언티스트
1,주니어 블록체인 데이터 사이언티스트,클로인트,클로인트(Kloint)는 디지털자산에 대한 다양한 위협으로부터 사회와 대중을 보호하...,"\n• 데이터 사이언스 관련 기업 근무, 연구활동 등 업무 수행 경험\n• 블록체인...",\n• 블록체인 및 암호화폐 관련 기업 근무 경력\n• 빅데이터 관련 기업 근무 경...,\nHadoop\nMongoDB\nMySQL\nSpark\nPython\nR\nTa...,https://www.wanted.co.kr/wd/174917,서울,데이터 사이언티스트
2,시니어 블록체인 데이터 사이언티스트,클로인트,클로인트(Kloint)는 디지털자산에 대한 다양한 위협으로부터 사회와 대중을 보호하...,\n• 데이터 사이언스 관련 실무 경력 7년 이상 또는 그에 준하는 역량 보유\n•...,\n• 블록체인 및 암호화폐 관련 기업 근무 경력\n• 빅데이터 관련 기업 근무 경...,\nHadoop\nMongoDB\nMySQL\nSpark\nPython\nR\nTa...,https://www.wanted.co.kr/wd/174913,서울,데이터 사이언티스트
3,데이터사이언티스트 (AI추천엔진팀),씨제이올리브영(CJ올리브영),"All Live Better,\n\nNo.1 헬스&뷰티 스토어 올리브영은 20년 이...","\n관련 업무 경력 5~10년 (데이터 상용화 모델링, 개발역량 등)\nMachin...","\n• 관련학과(수학, 통계, 전산, 데이터 분석) 전공자 우대\n• ML 관련 서...",\nAzure\nAWS\nGCP,https://www.wanted.co.kr/wd/164003,서울,데이터 사이언티스트
4,데이터 사이언티스트,씨제이대한통운(CJ대한통운),CJ대한통운은 글로벌 리딩 100년 기업으로의 도약을 목표로 성장하고 있는 종합 물...,\n[경력사항]\n• 데이터 모델링 경력 5년 이상\n\n,\n• 데이터 시각화 도구(Tableau 등) 사용이 가능하신 분\n• ML 프레임...,\nGitLab\nCSS\nTypeScript\nReact.js\nNext.js,https://www.wanted.co.kr/wd/163197,서울,데이터 사이언티스트


In [3]:
req = df.Requirement.tolist()

In [4]:
req2 = []
for idx in range(0, df.shape[0]):
    str = req[idx]
    new_str = re.sub('\n', '', str)
    new_str2 = re.sub(r'[^\uAC00-\uD7A30-9\s]', ' ', new_str)
    new_str3 = re.split(' ', new_str2)
    new_str4 = ' '.join(filter(None, new_str3))
    req2.append(new_str4)
    print(f'{idx}. {req2[idx]}')

0. 관련 업무 경력 3년 이상인 분 시계열 데이터를 분석하여 예측에 대한 경험이 있는 분 논문 등 알고리즘에 대해 분석 후 구현 경험이 있는 분 통계분석 인공지능 머신러닝 관련 석 박사학위 또는 이에 준하는 경력을 소지하신 분 언어 사용 가능
1. 데이터 사이언스 관련 기업 근무 연구활동 등 업무 수행 경험 블록체인 관련 기업 근무 연구활동 등 업무 수행 경험 데이터 분석 및 머신러닝 관련 언어 등 활용 역량 온체인 데이터 분석에 대한 이해도 보유자 선호
2. 데이터 사이언스 관련 실무 경력 7년 이상 또는 그에 준하는 역량 보유 대용량 데이터 분석 및 머신러닝 관련 언어 등 활용 능력 대용량 데이터 분석을 통해 문제를 정의하고 해결할 수 있는 능력 관련 알고리즘에 대한 논문 작성 등 분석 후 구현 경험 블록체인에 대한 비지니스적 기술적 이해도 보유자 선호 관련 실무경험 보유자 선호
3. 관련 업무 경력 5 10년 데이터 상용화 모델링 개발역량 등 활용 데이터 분석 가능 개발 가능자다양한 분석 툴 등 활용 가능고객 상품 브랜드 분석 및 비즈니스 관련 인사이트 도출 경험
4. 경력사항 데이터 모델링 경력 5년 이상
5. 관련 분야 경력 4년 이상이신 분 이론에 대한 전반적인 이해가 있으신 분 딥러닝 프레임워크 등 를 활용한 모델 개발 및 배포 경험이 있으신 분 대용량 데이터를 분석 및 시각화해본 경험이 있으신 분 해외 출장에 결격 사유가 없는 분 팀원들과 적극적으로 소통할 수 있는 분 주도적으로 업무를 이끌어가시는 분 제출서류 자율 양식의 이력서 필수 포함내용 성향 및 업무 가치관 개발경험 성공 및 실패 경험 협업 프로젝트 진행 관련 개발 성과 협업 경험을 보여줄 수 있는 포트폴리오
6. 데이터 사이언티스트로 이런 분을 찾아요 3년 이상의 모델링에 기반한 프로젝트 실무 경험이 있으신 분 개인화 추천 및 고객 경험 관련 모델링 경험이 있으신 분 이커머스 및 거래 데이터 도메인에 대한 이해도 혹은 관련 근무 경험이 있으신 분 과 프로그래밍을 능숙하게 다룰 수 있으

In [5]:
req2_df = pd.DataFrame({'req': req2})

In [6]:
req2_df.to_csv(path + 'req.csv', index=False, encoding='utf-8-sig')

In [7]:
req2_df.head()

,req
0,관련 업무 경력 3년 이상인 분 시계열 데이터를 분석하여 예측에 대한 경험이 있는 ...
1,데이터 사이언스 관련 기업 근무 연구활동 등 업무 수행 경험 블록체인 관련 기업 근...
2,데이터 사이언스 관련 실무 경력 7년 이상 또는 그에 준하는 역량 보유 대용량 데이...
3,관련 업무 경력 5 10년 데이터 상용화 모델링 개발역량 등 활용 데이터 분석 가능...
4,경력사항 데이터 모델링 경력 5년 이상


In [8]:
okt = Okt()

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.